In [ ]:
<img src='./figures/logo-ecole-polytechnique-ve.jpg' style='position:absolute; top:0; right:0;' width='100px' height='' alt='' />

<center><b>Bachelor of Ecole Polytechnique</b></center>
<center>MAA106: Introduction to Numerical Analysis, year 1, semester 2</center>
<center>Authors: Zhihui Li and Kevin Messali</center>
<br>
<center><i>Approximation of periodic functions</i></center>

# Approximation of periodic functions

&nbsp;

<table>
        <tr>
            <td><img src="./figures/Interpolation.png" alt="Interpolation" style="height: 300px;"/></td>
            <td><img src="./figures/LeastSquares.png" alt="LeastSquares" style="height: 290px;"/></td>
        </tr>
</table>

&nbsp;

In [3]:
import numpy
import scipy.integrate as integrate

def a_n(f, n):
    # We used scipy.integrate to calculate the integral
    # scipy.integrate() returns a tuple,
    # where the first element holding the estimated value of the integral
    # and the second element holding an upper bound on the error
    subresult = integrate.quad(lambda x: f(x)*numpy.cos(n*x), 0, numpy.pi)[0]
    return (1/numpy.pi)*subresult

def b_n(f, n):
    # We used scipy.integrate to calculate the integral
    # scipy.integrate() returns a tuple,
    # where the first element holding the estimated value of the integral
    # and the second element holding an upper bound on the error
    subresult = integrate.quad(lambda x: f(x)*numpy.sin(n*x), 0, numpy.pi)[0]
    return (1/numpy.pi)*subresult


def f1(x):
    return x**2

x = numpy.arange(-1,1,10)
n = 1
print(a_n(f1, n))
print(b_n(f1, n))

-2.0000000000000004
1.8683531088546308
